In [ ]:
"""
Created on Fri Jul 09 09:39 2021

This script is to transform the mean profiles to 2D fields to accelerate the tuning process

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import multimelt.useful_functions as uf
import multimelt.melt_functions as meltf
from multimelt.constants import *
from scipy import stats
from dask import delayed

import distributed
import glob

In [ ]:
%matplotlib qt5

In [ ]:
client = distributed.Client(n_workers=4,dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client

READ IN DATA

In [ ]:
nemo_run = 'bf663' # 'bf663','bi646' 

In [ ]:
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_profiles='/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_'+nemo_run+'/'
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'

plot_path = '/bettik/burgardc/PLOTS/NN_plots/Smith_formatting/'



In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

chunk_size = 700

Files for param

In [ ]:
file_other = xr.open_mfdataset(inputpath_data+'corrected_draft_bathy_isf.nc')
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim).chunk(chunks={'x': chunk_size, 'y': chunk_size})

In [ ]:
for tt in tqdm(range(1979, 2051)): #1970 + 35
    
    file_isf_orig = xr.open_mfdataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(tt)+'.nc').drop('time')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf).chunk(chunks={'x': chunk_size, 'y': chunk_size})

    ice_draft_pos = file_other_cut['corrected_isfdraft'].sel(time=tt).drop('time')
    ice_draft_isf = ice_draft_pos.where(file_isf['ISF_mask'] == file_isf.Nisf)

    depth_of_int = ice_draft_isf.where(ice_draft_isf<file_isf['front_bot_depth_max'], 
                                       file_isf['front_bot_depth_max']).chunk(chunks={'Nisf':1})  
    
    file_TS_orig = xr.open_mfdataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_'+str(tt)+'.nc', chunks={'Nisf': 1})
    file_TS = file_TS_orig.sel(Nisf=large_isf).chunk(chunks={'depth': 20})
    
    filled_TS = file_TS.ffill(dim='depth')
    
    T_isf = filled_TS['theta_ocean'].interp({'depth': depth_of_int}).drop('depth')
    
    #print('here1')
    T_isf = T_isf.where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')#.where(depth_of_int>0)
    T_isf = T_isf.to_dataset(name='theta_in')
    #print('here2')
    T_isf['salinity_in'] = filled_TS['salinity_ocean'].interp({'depth': depth_of_int}).drop('depth')
    T_isf['salinity_in'] = T_isf['salinity_in'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')#.where(depth_of_int>0)
    #print('here3')
    depth_of_int = depth_of_int.where(file_isf['ISF_mask'] == file_isf.Nisf)
    depth_of_int = depth_of_int.where(file_isf['ISF_mask'] > 1).sum('Nisf')
    depth_of_int = depth_of_int.where(depth_of_int>0)
    T_isf['freezing_T'] = meltf.freezing_temperature(T_isf['salinity_in'], -1*depth_of_int)
    #print('here4')
    T_isf['thermal_forcing'] = T_isf['theta_in'] - T_isf['freezing_T']
    T_isf['depth_of_int'] = depth_of_int
    T_isf = T_isf.where(T_isf['salinity_in']>0)
    
    
    T_isf.to_netcdf(inputpath_profiles+'T_S_2D_fields_isf_draft_oneFRIS_'+str(tt)+'.nc','w')
    del T_isf
    del depth_of_int